#### TOOLS, MODULES INSTALLATION

In [13]:
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
!pip install pyswarm

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
import numpy as np
import pandas as pd
import yfinance as yf
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.preprocessing import binarize, MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from pyswarm import pso

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### DATA PREPROCESSING - NORMALIZATION


In [16]:
# Load dataset
original_data = pd.read_csv('/content/drive/MyDrive/classes_NUS/Project_Module_1/preprocessed_data.csv')
original_data.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Date_x,Open,High,Low,Close,Adj Close,Volume,Ticker,Beta,P/E Ratio,...,PC1,PC2,PC3,Month,DayOfWeek,Daily_Return,MA20,MA50,Log_PE_Ratio,Log_PEG_Ratio
0,2020-01-02,-0.527490,-0.528801,-0.526355,-0.527421,-0.529219,-0.177086,Z74.SI,-0.33756,1.03557,...,-1.746289,-1.285688,0.711104,1,3,-0.000131,NaN,NaN,0.710776,-0.2276
1,2020-01-03,-0.527319,-0.528716,-0.526527,-0.527676,-0.529440,-0.103929,Z74.SI,-0.33756,1.03557,...,-1.746289,-1.285688,0.711104,1,4,0.000677,NaN,NaN,0.710776,-0.2276
2,2020-01-06,-0.527745,-0.529055,-0.526527,-0.527847,-0.529587,-0.133960,Z74.SI,-0.33756,1.03557,...,-1.746289,-1.285688,0.711104,1,0,0.000192,NaN,NaN,0.710776,-0.2276
3,2020-01-07,-0.527831,-0.529139,-0.526871,-0.527761,-0.529513,0.027768,Z74.SI,-0.33756,1.03557,...,-1.746289,-1.285688,0.711104,1,1,-0.000131,NaN,NaN,0.710776,-0.2276
4,2020-01-08,-0.528257,-0.528885,-0.526871,-0.527506,-0.529293,0.073270,Z74.SI,-0.33756,1.03557,...,-1.746289,-1.285688,0.711104,1,2,-0.001421,NaN,NaN,0.710776,-0.2276


In [17]:
# get the columns with the non numeric values
nncol = original_data.select_dtypes(exclude=['number'])
print(nncol.columns)

#drop the non numeric columns
col_drp = nncol.columns
temp_data = original_data.drop(columns = col_drp)

# get the columns with NaN values or Null values
if temp_data.isna().any().any():
  temp_data.fillna(0, inplace=True)

temp_data.describe()

Index(['Date_x', 'Ticker'], dtype='object')


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Open,High,Low,Close,Adj Close,Volume,Beta,P/E Ratio,Market Cap,52 Week High,...,PC1,PC2,PC3,Month,DayOfWeek,Daily_Return,MA20,MA50,Log_PE_Ratio,Log_PEG_Ratio
count,66343.000000,66343.000000,66343.000000,66343.000000,66343.000000,66343.000000,66343.000000,66343.000000,6.634300e+04,66343.000000,...,6.634300e+04,6.634300e+04,6.634300e+04,66343.000000,66343.000000,66343.000000,66343.000000,66343.000000,66343.000000,66343.000000
mean,-0.009757,-0.009441,-0.010087,-0.009760,-0.010201,-0.060291,-0.007953,-0.037481,2.786560e+11,94.899248,...,9.262128e-16,1.765031e-16,1.418022e-16,6.274392,2.015058,-0.001192,-0.009659,-0.009499,-0.375816,-0.169080
std,0.966994,0.968124,0.965806,0.966983,0.965142,0.570753,0.973993,0.847904,7.811223e+11,167.325227,...,1.000008e+00,1.000008e+00,1.000008e+00,3.380699,1.399944,0.367440,0.965551,0.963438,0.865194,0.431816
min,-0.556234,-0.557377,-0.555149,-0.556247,-0.554060,-0.328958,-1.433540,-0.880751,1.000000e-05,0.060000,...,-1.746289e+00,-1.285688e+00,-1.279092e+00,1.000000,0.000000,-67.852792,-0.556209,-0.556199,-2.126542,-1.257573
25%,-0.543084,-0.544188,-0.542019,-0.543099,-0.542182,-0.305161,-0.459101,-0.558389,1.556755e+09,1.580000,...,-6.457594e-02,-6.704259e-01,-1.008615e+00,3.000000,1.000000,-0.000294,-0.543061,-0.542935,-0.817326,-0.314209
50%,-0.525359,-0.526511,-0.524276,-0.525448,-0.525623,-0.262832,-0.154182,-0.182961,7.518913e+09,3.760000,...,3.617085e-01,-1.217801e-01,7.111045e-01,6.000000,2.000000,0.000026,-0.525341,-0.525230,-0.202068,-0.223999
75%,0.113090,0.116107,0.108516,0.112495,0.120898,-0.120878,0.476978,0.026823,2.066344e+10,112.790000,...,3.617085e-01,1.238477e+00,9.498384e-01,9.000000,3.000000,0.000499,0.117040,0.117593,0.026470,-0.096765
max,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.439320e+12,577.470000,...,1.188818e+00,1.238477e+00,9.498384e-01,12.000000,4.000000,15.927395,3.000000,3.000000,1.386294,1.386294


In [18]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
normalized_data = scaler.fit_transform(temp_data)
# the output from scaler normalisation is is ndarray, convert to pd
normalized_data = pd.DataFrame(normalized_data, columns=temp_data.columns)
normalized_data.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Open,High,Low,Close,Adj Close,Volume,Beta,P/E Ratio,Market Cap,52 Week High,...,PC1,PC2,PC3,Month,DayOfWeek,Daily_Return,MA20,MA50,Log_PE_Ratio,Log_PEG_Ratio
0,-0.535409,-0.536465,-0.534550,-0.535340,-0.537768,-0.204634,-0.338411,1.265543,-0.288668,-0.546779,...,-1.746289,-1.285688,0.711104,-1.56016,0.703563,0.002887,0.010004,0.009859,1.255903,-0.13552
1,-0.535232,-0.536377,-0.534728,-0.535604,-0.537996,-0.076458,-0.338411,1.265543,-0.288668,-0.546779,...,-1.746289,-1.285688,0.711104,-1.56016,1.417883,0.005086,0.010004,0.009859,1.255903,-0.13552
2,-0.535673,-0.536727,-0.534728,-0.535780,-0.538148,-0.129074,-0.338411,1.265543,-0.288668,-0.546779,...,-1.746289,-1.285688,0.711104,-1.56016,-1.439395,0.003766,0.010004,0.009859,1.255903,-0.13552
3,-0.535761,-0.536814,-0.535084,-0.535692,-0.538072,0.154286,-0.338411,1.265543,-0.288668,-0.546779,...,-1.746289,-1.285688,0.711104,-1.56016,-0.725076,0.002887,0.010004,0.009859,1.255903,-0.13552
4,-0.536202,-0.536552,-0.535084,-0.535428,-0.537844,0.234011,-0.338411,1.265543,-0.288668,-0.546779,...,-1.746289,-1.285688,0.711104,-1.56016,-0.010756,-0.000625,0.010004,0.009859,1.255903,-0.13552


#### DATA PREPROCESSING - BINARY DATA FOR APRIORI ALGORITHM


In [19]:
#Get transposed, binary data for Apriori algorithm
def process_data_binary_tranpose(normalized_data, original_data):

    temp = normalized_data
    temp['Ticker'] = original_data['Ticker']
    temp_gp = temp.groupby('Ticker').mean() #group by makes the Ticker as index and not an active column,
    temp_gp_bin = temp_gp.pct_change().apply(lambda x: (x > 0).astype(int))
    temp_gp_bin_tr_data = temp_gp_bin.T
    temp_gp_bin_tr_data.columns = temp_gp_bin.index
    temp_gp_bin_tr_data = temp_gp_bin_tr_data.reset_index() #need to apply reset_index for making use of the tickers rows (transpose done)
    temp_gp_bin_tr_data = temp_gp_bin_tr_data.drop(columns=['index'])
    binary_transposed_data = temp_gp_bin_tr_data

    return binary_transposed_data

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### GET THE CLUSTERS FOR PORTFOLIO CONSTRUCTION USING RULE BASED CLUSTERING ALGORITHM - APRIORI, ASSOCIATION RULES

In [20]:
#Apply Apriori algorithm for rule-based clustering
def rule_based_clusters(binary_transposed_data, min_support=0.43, min_confidence=0.43):

    frequent_itemsets = apriori(binary_transposed_data, min_support, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)
    clusters = {}
    index = 0
    for each_rule in rules.itertuples():
      if len(each_rule.consequents) > 2:
        print(each_rule.consequents)
        clusters[index] = list(each_rule.consequents)
        index = index + 1
      else:
        continue

    for key, value in clusters.items():
        print(f"{key}: {value}")

    return clusters

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### DIFFERENT TYPES OF PORTFOLIO CREATION FROM THE CLUSTERS

In [21]:
def create_portfolios(normalized_data_with_ticker, clusters, portfolio_type_weights = 'EW'):
    portfolios = {}
    # Iterate over each cluster
    for cluster_id, tickers in clusters.items():

        # Extract the stock tickers (values in the 'Ticker' column)
        cluster_data = normalized_data_with_ticker[normalized_data_with_ticker['Ticker'].isin(tickers)]
        cluster_data_n = cluster_data.drop(columns=['Ticker'])
        num_ticker = len(tickers)
        # Equal Weighted (EW) Portfolio
        if portfolio_type_weights == 'EW':
          ew_weights = np.ones(num_ticker) / num_ticker
          portfolio_name = f'EW_Cluster_{cluster_id}'
          portfolios[portfolio_name] = dict(zip(tickers, ew_weights))

        elif portfolio_type_weights == 'MCW':
          # Market Cap Weighted (MCW) Portfolio (Simulated Market Cap for simplicity)
          market_caps = np.random.uniform(100, 500, num_ticker)  # Simulate market caps
          mcw_weights = market_caps / np.sum(market_caps)
          portfolio_name = f'MCW_Cluster_{cluster_id}'
          portfolios[portfolio_name] = mcw_weights
          portfolios[portfolio_name] = dict(zip(tickers, mcw_weights))

        elif portfolio_type_weights == 'GMV':
          # Covariance Matrix (from normalized_data)
          returns = cluster_data_n.pct_change().dropna()
          cov_matrix = returns.cov()
          # Global Minimum Variance (GMV) Portfolio
          inv_cov_matrix = np.linalg.inv(cov_matrix)
          ones = np.ones(cov_matrix.shape[0])
          gmv_weights = np.dot(inv_cov_matrix, ones) / np.dot(ones.T, np.dot(inv_cov_matrix, ones))
          portfolio_name = f'GMV_Cluster_{cluster_id}'
          portfolios[portfolio_name] = dict(zip(tickers, gmv_weights))

        elif portfolio_type_weights == 'MSR':
          # Maximum Sharpe Ratio (MSR) Portfolio
          cluster_n = cluster_data_n.pct_change().dropna()
          cluster_exp_returns = cluster_n.mean() * 252  # Annualized returns
          cluster_exp_returns = pd.DataFrame(cluster_exp_returns)
          cov_matrix = cluster_n.cov()
          inv_cov_matrix = np.linalg.inv(cov_matrix)
          ones = np.ones((len(cluster_exp_returns), 1))
          numer = np.dot(inv_cov_matrix, cluster_exp_returns)
          denom = np.dot(ones.T, numer)
          msr_weights = numer / denom
          portfolio_name = f'MSR_Cluster_{cluster_id}'
          portfolios[portfolio_name] = dict(zip(tickers, msr_weights.flatten()))
        else:
            print("Invalid portfolio_type_weights set")
            return None
    return portfolios

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### OBJECTIVE FUNCTION

In [22]:
# 5. Define objective functions for PSO optimization
def objective_function(weights, returns, cov_matrix, risk_free_rate=0.01):

    #portfolio_return = np.dot(weights, returns)
    weights = weights.flatten()

    if isinstance(returns, (int, float)):  # Check if returns is a scalar
      portfolio_return = returns * weights[0]
      portfolio_volatility = np.sqrt(cov_matrix * (weights[0]**2))  # Assume cov_matrix is also a scalar
      # Modify calculation for Sharpe Ratio or other metric as needed
    else:
      # Portfolio returns
      portfolio_return = np.sum(returns * weights)
      # Portfolio volatility (risk)
      portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

    if portfolio_volatility == 0:
        portfolio_volatility = np.inf  # Avoid division by zero

    # Sharpe Ratio
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility

    # We want to maximize Sharpe Ratio and minimize volatility
    return sharpe_ratio#, #portfolio_volatility

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### PSO OPTIMISATION


In [23]:
# 6. Optimize portfolio using PSO
def pso_optimization(returns, cov_matrix):
    num_assets = len(returns)
    bounds = [(0, 1) for _ in range(num_assets)]
    lb = [0] * num_assets
    ub = [1] * num_assets
    optimal_weights, _ = pso(objective_function, lb, ub,
                             args=(returns.values, cov_matrix.values), # Pass returns and cov_matrix as NumPy arrays
                             swarmsize=10, maxiter=10)

    # Normalize weights to sum to 1
    optimal_weights /= np.sum(optimal_weights)

    return optimal_weights

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### LSTM FORECAST

In [27]:


# LSTM for stock price forecasting
def lstm_forecast(prices, portfolio_cluster, look_back=60):

    # Scaling data between 0 and 1 for LSTM
    dataset = prices.values  # Assuming 'prices' is your DataFrame
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)

    # Preparing the data for LSTM
    X, y = [], []
    for i in range(look_back, len(dataset)):
        X.append(dataset[i - look_back:i])  # Sequence of past 'look_back' days
        y.append(dataset[i])  # Predict next value based on 'look_back' values

    X, y = np.array(X), np.array(y)

    # Get the number of features (columns) in your prices data
    num_features = prices.shape[1]

    # Reshape X to the required shape for LSTM input (samples, time steps, features)
    X = X.reshape(X.shape[0], X.shape[1], num_features)

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(units=10, return_sequences=True, input_shape=(X_train.shape[1], num_features)))  # Update the input shape to match number of features
    model.add(Dropout(0.2))
    model.add(LSTM(units=10, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=num_features))  # Predicting the same number of features as input

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=3, batch_size=16)

    # Predicting future prices
    predicted_prices = model.predict(X_test)

    # Inverse scaling to get the actual prices
    predicted_prices = scaler.inverse_transform(predicted_prices)

    model.save_weights('model_weights_' + portfolio_cluster + '.weights.h5')

    return predicted_prices

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### MAIN API - ACTIVE PORTFOLIO OPTIMISATION

In [28]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def active_portfolio_optimization(normalized_data, original_data,portfolio_type_weights = 'EW'):

    # Get clusters from rule based algorithm
    binary_data = process_data_binary_tranpose(normalized_data, original_data)
    clusters = rule_based_clusters(binary_data)

    #create portfolios from clusters based on type, here default is equally weighted portfolio - EW
    normalized_data_with_ticker = normalized_data.copy()
    normalized_data_with_ticker['Ticker'] = original_data['Ticker']
    portfolios = create_portfolios(normalized_data_with_ticker, clusters, portfolio_type_weights)
    for key, value in portfolios.items():
      print(f"{key}: {value}")
    # one dummy output - EW_Cluster_0: {'K71U.SI': 0.3333333333333333, 'OXMU.SI': 0.3333333333333333, 'C52.SI': 0.3333333333333333}

    #optimise the portfolios created
    optimized_portfolios = {}
    metrics = {}
    for portfolio_cluster, ticker_weights in portfolios.items():
        tickers_in_each_cluster = list(ticker_weights.keys())
        print(f"Processing portfolio: {portfolio_cluster} with tickers: {tickers_in_each_cluster}")
        #cluster_prices = original_data['Ticker'].isin(tickers_in_each_cluster)  # Prices of the stocks in this portfolio cluster
        cluster_prices = original_data.loc[original_data['Ticker'].isin(tickers_in_each_cluster), 'Adj Close']
        cluster_prices = cluster_prices.values.reshape(-1,1)
        cluster_prices = pd.DataFrame(cluster_prices)
        print(f"The first rows in cluster_prices : {cluster_prices.head()}")
        # Predict future prices using LSTM
        forecasted_prices = lstm_forecast(cluster_prices, portfolio_cluster)
        forecasted_prices = np.resize(forecasted_prices, cluster_prices.shape)
        print(f"The lstm_forecast prices : {forecasted_prices}")
        # Calculate returns and covariance matrix for PSO optimization
        returns = cluster_prices.pct_change().mean() * 252  # Annualized returns
        cov_matrix = cluster_prices.pct_change().cov()

        # Optimize the portfolio using PSO
        optimal_weights = pso_optimization(returns, cov_matrix)

        # Calculate accuracy metrics for LSTM forecast
        mae = mean_absolute_error(cluster_prices, forecasted_prices)
        mse = mean_squared_error(cluster_prices, forecasted_prices)
        rmse = np.sqrt(mse)
        mape = np.mean(np.abs((cluster_prices - forecasted_prices) / cluster_prices)) * 100  # MAPE in percentage

        # Store the metrics for this portfolio
        metrics[portfolio_cluster] = {
            'MAE': mae,
            'MSE': mse,
            'RMSE': rmse,
            'MAPE': mape
        }

        # Store the optimized portfolio and forecasted prices
        optimized_portfolios[portfolio_cluster] = {
            'weights': optimal_weights,
            'forecasted_prices': forecasted_prices,
            'metrics': metrics[portfolio_cluster]  # Attach metrics
        }

        # Print the optimized weights and forecasted prices
        print(f"Optimized Weights for {portfolio_cluster}: {optimal_weights}")
        print(f"Forecasted Prices for {portfolio_cluster}: {forecasted_prices}")
        print(f"Metrics for {portfolio_cluster}: {metrics[portfolio_cluster]}")

    return optimized_portfolios

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Result


- CLUSTERS created
- 0: ['C52.SI', 'C2PU.SI', 'K71U.SI']
- 1: ['OXMU.SI', 'C52.SI', 'K71U.SI'] -> best cluster based on metrics
- 2: ['OXMU.SI', 'C2PU.SI', 'K71U.SI']
- 3: ['OXMU.SI', 'C2PU.SI', 'C52.SI']

In [29]:
optimized_portfolios_set1 = active_portfolio_optimization(normalized_data, original_data, portfolio_type_weights = 'EW')


frozenset({'OXMU.SI', 'K71U.SI', 'C2PU.SI'})
frozenset({'K71U.SI', 'C52.SI', 'C2PU.SI'})
frozenset({'OXMU.SI', 'C52.SI', 'C2PU.SI'})
frozenset({'K71U.SI', 'OXMU.SI', 'C52.SI'})
0: ['OXMU.SI', 'K71U.SI', 'C2PU.SI']
1: ['K71U.SI', 'C52.SI', 'C2PU.SI']
2: ['OXMU.SI', 'C52.SI', 'C2PU.SI']
3: ['K71U.SI', 'OXMU.SI', 'C52.SI']
EW_Cluster_0: {'OXMU.SI': 0.3333333333333333, 'K71U.SI': 0.3333333333333333, 'C2PU.SI': 0.3333333333333333}
EW_Cluster_1: {'K71U.SI': 0.3333333333333333, 'C52.SI': 0.3333333333333333, 'C2PU.SI': 0.3333333333333333}
EW_Cluster_2: {'OXMU.SI': 0.3333333333333333, 'C52.SI': 0.3333333333333333, 'C2PU.SI': 0.3333333333333333}
EW_Cluster_3: {'K71U.SI': 0.3333333333333333, 'OXMU.SI': 0.3333333333333333, 'C52.SI': 0.3333333333333333}
Processing portfolio: EW_Cluster_0 with tickers: ['OXMU.SI', 'K71U.SI', 'C2PU.SI']
The first rows in cluster_prices :           0
0 -0.545947
1 -0.546014
2 -0.545947
3 -0.545947
4 -0.545880
Epoch 1/3


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0978
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0102
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.0089
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
The lstm_forecast prices : [[-0.54972136]
 [-0.5497275 ]
 [-0.54973227]
 ...
 [-0.54983324]
 [-0.549828  ]
 [-0.5498242 ]]
Stopping search: maximum iterations reached --> 10
Optimized Weights for EW_Cluster_0: [1.]
Forecasted Prices for EW_Cluster_0: [[-0.54972136]
 [-0.5497275 ]
 [-0.54973227]
 ...
 [-0.54983324]
 [-0.549828  ]
 [-0.5498242 ]]
Metrics for EW_Cluster_0: {'MAE': 0.011387106535730877, 'MSE': 0.0002911240801546535, 'RMSE': 0.017062358575374435, 'MAPE': 2.1677258003876756}
Processing portfolio: EW_Cluster_1 with tickers: ['K71U.SI', 'C52.SI', 'C2PU.SI']
The first rows in cluster_prices :           0
0 -0.536907
1 -0.536980
2 -0.537272
3 -0.537126
4 -0.537418
Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 0.0709
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0053
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0042
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
The lstm_forecast prices : [[-0.5175321 ]
 [-0.51744485]
 [-0.51737076]
 ...
 [-0.51753914]
 [-0.5176301 ]
 [-0.5177134 ]]
Stopping search: maximum iterations reached --> 10
Optimized Weights for EW_Cluster_1: [1.]
Forecasted Prices for EW_Cluster_1: [[-0.5175321 ]
 [-0.51744485]
 [-0.51737076]
 ...
 [-0.51753914]
 [-0.5176301 ]
 [-0.5177134 ]]
Metrics for EW_Cluster_1: {'MAE': 0.016999908662738268, 'MSE': 0.0004009407064010549, 'RMSE': 0.02002350384925313, 'MAPE': 3.1268325056372253}
Processing portfolio: EW_Cluster_2 with tickers: ['OXMU.SI', 'C52.SI', 'C2PU.SI']
The first rows in cluster_prices :           0
0 -0.536907
1 -0.536980
2 -0.537272
3 -0.537126
4 -0.537418
Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.0585
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0094
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0092
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
The lstm_forecast prices : [[-0.5495803 ]
 [-0.54958636]
 [-0.54959166]
 ...
 [-0.5497078 ]
 [-0.54970145]
 [-0.5496959 ]]
Stopping search: maximum iterations reached --> 10
Optimized Weights for EW_Cluster_2: [1.]
Forecasted Prices for EW_Cluster_2: [[-0.5495803 ]
 [-0.54958636]
 [-0.54959166]
 ...
 [-0.5497078 ]
 [-0.54970145]
 [-0.5496959 ]]
Metrics for EW_Cluster_2: {'MAE': 0.01262789550862873, 'MSE': 0.00030678131672774326, 'RMSE': 0.0175151738994434, 'MAPE': 2.3982371733372565}
Processing portfolio: EW_Cluster_3 with tickers: ['K71U.SI', 'OXMU.SI', 'C52.SI']
The first rows in cluster_prices :           0
0 -0.536907
1 -0.536980
2 -0.537272
3 -0.537126
4 -0.537418
Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0771
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0089
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0071
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
The lstm_forecast prices : [[-0.5498464 ]
 [-0.5498522 ]
 [-0.54985744]
 ...
 [-0.54999524]
 [-0.5499915 ]
 [-0.5499879 ]]
Stopping search: maximum iterations reached --> 10
Optimized Weights for EW_Cluster_3: [1.]
Forecasted Prices for EW_Cluster_3: [[-0.5498464 ]
 [-0.5498522 ]
 [-0.54985744]
 ...
 [-0.54999524]
 [-0.5499915 ]
 [-0.5499879 ]]
Metrics for EW_Cluster_3: {'MAE': 0.004657112110393389, 'MSE': 3.2552675393068766e-05, 'RMSE': 0.005705495192625156, 'MAPE': 0.8553877884080616}


- Metrics for EW_Cluster_3: {'MAE': 0.004657112110393389, 'MSE': 3.2552675393068766e-05, 'RMSE': 0.005705495192625156, 'MAPE': 0.8553877884080616}

In [30]:
optimized_portfolios_set1 = active_portfolio_optimization(normalized_data, original_data, portfolio_type_weights = 'MCW')

frozenset({'OXMU.SI', 'K71U.SI', 'C2PU.SI'})
frozenset({'K71U.SI', 'C52.SI', 'C2PU.SI'})
frozenset({'OXMU.SI', 'C52.SI', 'C2PU.SI'})
frozenset({'K71U.SI', 'OXMU.SI', 'C52.SI'})
0: ['OXMU.SI', 'K71U.SI', 'C2PU.SI']
1: ['K71U.SI', 'C52.SI', 'C2PU.SI']
2: ['OXMU.SI', 'C52.SI', 'C2PU.SI']
3: ['K71U.SI', 'OXMU.SI', 'C52.SI']
MCW_Cluster_0: {'OXMU.SI': 0.5399629631865727, 'K71U.SI': 0.16200208637605615, 'C2PU.SI': 0.29803495043737116}
MCW_Cluster_1: {'K71U.SI': 0.3552221546553253, 'C52.SI': 0.3396449291978439, 'C2PU.SI': 0.30513291614683086}
MCW_Cluster_2: {'OXMU.SI': 0.3271165235086724, 'C52.SI': 0.416590330152011, 'C2PU.SI': 0.25629314633931655}
MCW_Cluster_3: {'K71U.SI': 0.3459780986862702, 'OXMU.SI': 0.36653566994865966, 'C52.SI': 0.2874862313650701}
Processing portfolio: MCW_Cluster_0 with tickers: ['OXMU.SI', 'K71U.SI', 'C2PU.SI']
The first rows in cluster_prices :           0
0 -0.545947
1 -0.546014
2 -0.545947
3 -0.545947
4 -0.545880


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.1039
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.0116
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0087
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
The lstm_forecast prices : [[-0.54949486]
 [-0.5495016 ]
 [-0.5495079 ]
 ...
 [-0.5496486 ]
 [-0.54964167]
 [-0.54963475]]
Stopping search: maximum iterations reached --> 10
Optimized Weights for MCW_Cluster_0: [1.]
Forecasted Prices for MCW_Cluster_0: [[-0.54949486]
 [-0.5495016 ]
 [-0.5495079 ]
 ...
 [-0.5496486 ]
 [-0.54964167]
 [-0.54963475]]
Metrics for MCW_Cluster_0: {'MAE': 0.011474026963815422, 'MSE': 0.0002940210304795461, 'RMSE': 0.01714704144975296, 'MAPE': 2.1839669026522004}
Processing portfolio: MCW_Cluster_1 with tickers: ['K71U.SI', 'C52.SI', 'C2PU.SI']
The first rows in cluster_prices :           0
0 -0.536907
1 -0.536980
2 -0.537272
3 -0.537126
4 -0.537418
Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 0.0285
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0041
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0032
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
The lstm_forecast prices : [[-0.5194253 ]
 [-0.51932967]
 [-0.51925737]
 ...
 [-0.5200468 ]
 [-0.5201554 ]
 [-0.5202493 ]]
Stopping search: maximum iterations reached --> 10
Optimized Weights for MCW_Cluster_1: [1.]
Forecasted Prices for MCW_Cluster_1: [[-0.5194253 ]
 [-0.51932967]
 [-0.51925737]
 ...
 [-0.5200468 ]
 [-0.5201554 ]
 [-0.5202493 ]]
Metrics for MCW_Cluster_1: {'MAE': 0.01557938991010045, 'MSE': 0.0003270081149540015, 'RMSE': 0.01808336569762392, 'MAPE': 2.8685877392279973}
Processing portfolio: MCW_Cluster_2 with tickers: ['OXMU.SI', 'C52.SI', 'C2PU.SI']
The first rows in cluster_prices :           0
0 -0.536907
1 -0.536980
2 -0.537272
3 -0.537126
4 -0.537418
Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0908
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.0120
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0085
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
The lstm_forecast prices : [[-0.5497293 ]
 [-0.5497365 ]
 [-0.54974234]
 ...
 [-0.549864  ]
 [-0.5498558 ]
 [-0.54984915]]
Stopping search: maximum iterations reached --> 10
Optimized Weights for MCW_Cluster_2: [1.]
Forecasted Prices for MCW_Cluster_2: [[-0.5497293 ]
 [-0.5497365 ]
 [-0.54974234]
 ...
 [-0.549864  ]
 [-0.5498558 ]
 [-0.54984915]]
Metrics for MCW_Cluster_2: {'MAE': 0.012825018899157197, 'MSE': 0.0003145862928722992, 'RMSE': 0.01773658064206005, 'MAPE': 2.435354312932396}
Processing portfolio: MCW_Cluster_3 with tickers: ['K71U.SI', 'OXMU.SI', 'C52.SI']
The first rows in cluster_prices :           0
0 -0.536907
1 -0.536980
2 -0.537272
3 -0.537126
4 -0.537418
Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 0.0900
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0103
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0075
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
The lstm_forecast prices : [[-0.54994696]
 [-0.5499529 ]
 [-0.5499585 ]
 ...
 [-0.5501033 ]
 [-0.5501    ]
 [-0.5500967 ]]
Stopping search: maximum iterations reached --> 10
Optimized Weights for MCW_Cluster_3: [1.]
Forecasted Prices for MCW_Cluster_3: [[-0.54994696]
 [-0.5499529 ]
 [-0.5499585 ]
 ...
 [-0.5501033 ]
 [-0.5501    ]
 [-0.5500967 ]]
Metrics for MCW_Cluster_3: {'MAE': 0.0048060208658446246, 'MSE': 3.4582090096632244e-05, 'RMSE': 0.00588065388342421, 'MAPE': 0.882730260343261}


- Metrics for MCW_Cluster_3: {'MAE': 0.0048060208658446246, 'MSE': 3.4582090096632244e-05, 'RMSE': 0.00588065388342421, 'MAPE': 0.882730260343261}

In [31]:
optimized_portfolios_set1 = active_portfolio_optimization(normalized_data, original_data, portfolio_type_weights = 'GMV')

frozenset({'OXMU.SI', 'K71U.SI', 'C2PU.SI'})
frozenset({'K71U.SI', 'C52.SI', 'C2PU.SI'})
frozenset({'OXMU.SI', 'C52.SI', 'C2PU.SI'})
frozenset({'K71U.SI', 'OXMU.SI', 'C52.SI'})
0: ['OXMU.SI', 'K71U.SI', 'C2PU.SI']
1: ['K71U.SI', 'C52.SI', 'C2PU.SI']
2: ['OXMU.SI', 'C52.SI', 'C2PU.SI']
3: ['K71U.SI', 'OXMU.SI', 'C52.SI']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


GMV_Cluster_0: {'OXMU.SI': 3.0158880825412335e-15, 'K71U.SI': 8.319378731199472e-16, 'C2PU.SI': 9.711918938984257e-16}
GMV_Cluster_1: {'K71U.SI': -3.366436271514299e-17, 'C52.SI': -4.512797562569167e-15, 'C2PU.SI': -4.321775796660673e-15}
GMV_Cluster_2: {'OXMU.SI': 6.813276498767571e-15, 'C52.SI': -5.679386269279177e-15, 'C2PU.SI': -7.713770162093047e-15}
GMV_Cluster_3: {'K71U.SI': 9.017911069511968e-16, 'OXMU.SI': 6.143106534793197e-16, 'C52.SI': -3.5132977726386065e-15}
Processing portfolio: GMV_Cluster_0 with tickers: ['OXMU.SI', 'K71U.SI', 'C2PU.SI']
The first rows in cluster_prices :           0
0 -0.545947
1 -0.546014
2 -0.545947
3 -0.545947
4 -0.545880
Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.1812
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0132
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0120
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
The lstm_forecast prices : [[-0.5485097 ]
 [-0.54851437]
 [-0.54851836]
 ...
 [-0.54861104]
 [-0.5486065 ]
 [-0.5486026 ]]
Stopping search: maximum iterations reached --> 10
Optimized Weights for GMV_Cluster_0: [1.]
Forecasted Prices for GMV_Cluster_0: [[-0.5485097 ]
 [-0.54851437]
 [-0.54851836]
 ...
 [-0.54861104]
 [-0.5486065 ]
 [-0.5486026 ]]
Metrics for GMV_Cluster_0: {'MAE': 0.01069468493516153, 'MSE': 0.0002623129650087251, 'RMSE': 0.016196078692347883, 'MAPE': 2.0366090480105536}
Processing portfolio: GMV_Cluster_1 with tickers: ['K71U.SI', 'C52.SI', 'C2PU.SI']
The first rows in cluster_prices :           0
0 -0.536907
1 -0.536980
2 -0.537272
3 -0.537126
4 -0.537418
Epoch 1/3


...

173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0329
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0045
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0031
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
The lstm_forecast prices : [[-0.51652855]
 [-0.5164015 ]
 [-0.5162976 ]
 ...
 [-0.5171287 ]
 [-0.517272  ]
 [-0.51740074]]
Stopping search: maximum iterations reached --> 10
Optimized Weights for GMV_Cluster_1: [1.]
Forecasted Prices for GMV_Cluster_1: [[-0.51652855]
 [-0.5164015 ]
 [-0.5162976 ]
 ...
 [-0.5171287 ]
 [-0.517272  ]
 [-0.51740074]]
Metrics for GMV_Cluster_1: {'MAE': 0.016843194349840124, 'MSE': 0.00039548073004549186, 'RMSE': 0.01988669731366905, 'MAPE': 3.098462739699639}
Processing portfolio: GMV_Cluster_2 with tickers: ['OXMU.SI', 'C52.SI', 'C2PU.SI']
The first rows in cluster_prices :           0
0 -0.536907
1 -0.536980
2 -0.537272
3 -0.537126
4 -0.537418
Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.1379
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0131
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0092
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
The lstm_forecast prices : [[-0.54937345]
 [-0.5493785 ]
 [-0.5493833 ]
 ...
 [-0.54949206]
 [-0.54948777]
 [-0.54948324]]
Stopping search: maximum iterations reached --> 10
Optimized Weights for GMV_Cluster_2: [1.]
Forecasted Prices for GMV_Cluster_2: [[-0.54937345]
 [-0.5493785 ]
 [-0.5493833 ]
 ...
 [-0.54949206]
 [-0.54948777]
 [-0.54948324]]
Metrics for GMV_Cluster_2: {'MAE': 0.012376293684637297, 'MSE': 0.000296280539411704, 'RMSE': 0.01721280161425513, 'MAPE': 2.3507043999801818}
Processing portfolio: GMV_Cluster_3 with tickers: ['K71U.SI', 'OXMU.SI', 'C52.SI']
The first rows in cluster_prices :           0
0 -0.536907
1 -0.536980
2 -0.537272
3 -0.537126
4 -0.537418
Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.0569
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0084
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0070
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
The lstm_forecast prices : [[-0.54997367]
 [-0.5499799 ]
 [-0.54998535]
 ...
 [-0.5501153 ]
 [-0.5501105 ]
 [-0.5501064 ]]
Stopping search: maximum iterations reached --> 10
Optimized Weights for GMV_Cluster_3: [1.]
Forecasted Prices for GMV_Cluster_3: [[-0.54997367]
 [-0.5499799 ]
 [-0.54998535]
 ...
 [-0.5501153 ]
 [-0.5501105 ]
 [-0.5501064 ]]
Metrics for GMV_Cluster_3: {'MAE': 0.0047469111093427245, 'MSE': 3.3709590026082156e-05, 'RMSE': 0.005805996040825567, 'MAPE': 0.8718782356212688}


- Metrics for GMV_Cluster_3: {'MAE': 0.0047469111093427245, 'MSE': 3.3709590026082156e-05, 'RMSE': 0.005805996040825567, 'MAPE': 0.8718782356212688}

In [32]:
optimized_portfolios_set1 = active_portfolio_optimization(normalized_data, original_data, portfolio_type_weights = 'MSR')

frozenset({'OXMU.SI', 'K71U.SI', 'C2PU.SI'})
frozenset({'K71U.SI', 'C52.SI', 'C2PU.SI'})
frozenset({'OXMU.SI', 'C52.SI', 'C2PU.SI'})
frozenset({'K71U.SI', 'OXMU.SI', 'C52.SI'})
0: ['OXMU.SI', 'K71U.SI', 'C2PU.SI']
1: ['K71U.SI', 'C52.SI', 'C2PU.SI']
2: ['OXMU.SI', 'C52.SI', 'C2PU.SI']
3: ['K71U.SI', 'OXMU.SI', 'C52.SI']
MSR_Cluster_0: {'OXMU.SI': -0.004010857786348803, 'K71U.SI': -0.021696935608734605, 'C2PU.SI': -0.013965208259085463}
MSR_Cluster_1: {'K71U.SI': -0.0006948749674130618, 'C52.SI': 0.0027647140595214886, 'C2PU.SI': 0.0024170604552142534}
MSR_Cluster_2: {'OXMU.SI': -0.0009750903144902756, 'C52.SI': 0.017754646807605277, 'C2PU.SI': 0.013069133098175834}
MSR_Cluster_3: {'K71U.SI': 0.00438154914916516, 'OXMU.SI': 0.05776671445403798, 'C52.SI': 0.0030716394468776253}
Processing portfolio: MSR_Cluster_0 with tickers: ['OXMU.SI', 'K71U.SI', 'C2PU.SI']
The first rows in cluster_prices :           0
0 -0.545947
1 -0.546014
2 -0.545947
3 -0.545947
4 -0.545880


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0849
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.0122
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.0096
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
The lstm_forecast prices : [[-0.54937947]
 [-0.5493855 ]
 [-0.54939044]
 ...
 [-0.54949194]
 [-0.54948515]
 [-0.5494794 ]]
Stopping search: maximum iterations reached --> 10
Optimized Weights for MSR_Cluster_0: [1.]
Forecasted Prices for MSR_Cluster_0: [[-0.54937947]
 [-0.5493855 ]
 [-0.54939044]
 ...
 [-0.54949194]
 [-0.54948515]
 [-0.5494794 ]]
Metrics for MSR_Cluster_0: {'MAE': 0.011195470286678735, 'MSE': 0.00028378179939527885, 'RMSE': 0.016845824390491516, 'MAPE': 2.131618137206828}
Processing portfolio: MSR_Cluster_1 with tickers: ['K71U.SI', 'C52.SI', 'C2PU.SI']
The first rows in cluster_prices :           0
0 -0.536907
1 -0.536980
2 -0.537272
3 -0.537126
4 -0.537418
Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0463
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0060
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0048
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
The lstm_forecast prices : [[-0.5183275 ]
 [-0.51822025]
 [-0.51812464]
 ...
 [-0.51827294]
 [-0.5183808 ]
 [-0.5184835 ]]
Stopping search: maximum iterations reached --> 10
Optimized Weights for MSR_Cluster_1: [1.]
Forecasted Prices for MSR_Cluster_1: [[-0.5183275 ]
 [-0.51822025]
 [-0.51812464]
 ...
 [-0.51827294]
 [-0.5183808 ]
 [-0.5184835 ]]
Metrics for MSR_Cluster_1: {'MAE': 0.01630098227747668, 'MSE': 0.000367178033934811, 'RMSE': 0.019161890145150372, 'MAPE': 2.999173157878692}
Processing portfolio: MSR_Cluster_2 with tickers: ['OXMU.SI', 'C52.SI', 'C2PU.SI']
The first rows in cluster_prices :           0
0 -0.536907
1 -0.536980
2 -0.537272
3 -0.537126
4 -0.537418
Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.1708
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0121
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0110
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
The lstm_forecast prices : [[-0.5491844 ]
 [-0.5491905 ]
 [-0.5491955 ]
 ...
 [-0.54929847]
 [-0.54929185]
 [-0.5492865 ]]
Stopping search: maximum iterations reached --> 10
Optimized Weights for MSR_Cluster_2: [1.]
Forecasted Prices for MSR_Cluster_2: [[-0.5491844 ]
 [-0.5491905 ]
 [-0.5491955 ]
 ...
 [-0.54929847]
 [-0.54929185]
 [-0.5492865 ]]
Metrics for MSR_Cluster_2: {'MAE': 0.012346182269007367, 'MSE': 0.0002948079190988218, 'RMSE': 0.017169971435585494, 'MAPE': 2.3449563299193326}
Processing portfolio: MSR_Cluster_3 with tickers: ['K71U.SI', 'OXMU.SI', 'C52.SI']
The first rows in cluster_prices :           0
0 -0.536907
1 -0.536980
2 -0.537272
3 -0.537126
4 -0.537418
Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0379
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0076
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.0065
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
The lstm_forecast prices : [[-0.55009645]
 [-0.5501033 ]
 [-0.55010927]
 ...
 [-0.5502351 ]
 [-0.55022836]
 [-0.5502223 ]]
Stopping search: maximum iterations reached --> 10
Optimized Weights for MSR_Cluster_3: [1.]
Forecasted Prices for MSR_Cluster_3: [[-0.55009645]
 [-0.5501033 ]
 [-0.55010927]
 ...
 [-0.5502351 ]
 [-0.55022836]
 [-0.5502223 ]]
Metrics for MSR_Cluster_3: {'MAE': 0.004838376151140671, 'MSE': 3.4829945859603045e-05, 'RMSE': 0.005901690085018277, 'MAPE': 0.8886607884233877}


- Metrics for MSR_Cluster_3: {'MAE': 0.004838376151140671, 'MSE': 3.4829945859603045e-05, 'RMSE': 0.005901690085018277, 'MAPE': 0.8886607884233877}

#### Results and analysis for experiment - rule based clustering with PSO optimization :
- Cluster 3 best here

## Max return
- Processing portfolio: GMV_Cluster_3 with tickers: ['K71U.SI', 'OXMU.SI', 'C52.SI']
- max return - /content/model_weights_GMV_Cluster_3.weights.h5
- max return - GMV weights - GMV_Cluster_3: {'K71U.SI': 9.017911069511968e-16, 'OXMU.SI': 6.143106534793197e-16, 'C52.SI': -3.5132977726386065e-15}

## Min risk
- Processing portfolio: MCW_Cluster_3 with tickers: ['K71U.SI', 'OXMU.SI', 'C52.SI']
- min risk - /content/model_weights_MCW_Cluster_3.weights.h5
- min risk - MCW weights - MCW_Cluster_3: {'K71U.SI': 0.3459780986862702, 'OXMU.SI': 0.36653566994865966, 'C52.SI': 0.2874862313650701}
